In [47]:
from src.mongo import connectCollection
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import json
import folium
from folium import plugins
from folium.plugins import MarkerCluster

# Queries collection

In [69]:
db, coll = connectCollection('companies','queries')

In [70]:
queries = db.queries.find({ 
    "score":{
        "$lte": 4
    }})

In [71]:
locations = pd.DataFrame(queries)

In [72]:
locations.head()

,_id,name,country,city,coordinates,longitude,latitude,category,score
0,5e0e60a33697832754a1ac31,Starbucks,USA,New York,"{'type': 'Point', 'coordinates': [-73.98512676...",-73.985127,40.759563,Coffee Shop,1.0
1,5e0e60a33697832754a1ac32,Starbucks,USA,New York,"{'type': 'Point', 'coordinates': [-73.99094694...",-73.990947,40.756638,Coffee Shop,2.0
2,5e0e60a33697832754a1ac33,Starbucks,USA,New York,"{'type': 'Point', 'coordinates': [-73.98380932...",-73.983809,40.760567,Coffee Shop,2.0
3,5e0e60a33697832754a1ac34,Starbucks,USA,New York,"{'type': 'Point', 'coordinates': [-73.98704612...",-73.987046,40.754292,Coffee Shop,2.0
4,5e0e60a33697832754a1ac35,Starbucks,USA,New York,"{'type': 'Point', 'coordinates': [-73.98363055...",-73.983631,40.762111,Coffee Shop,2.0


In [73]:
locations.category.unique()

array(['Coffee Shop', 'Airport', 'Vegan', 'School'], dtype=object)

In [111]:
colors = {1 : 'red', 2 : 'yellow', 3: 'blue', 4: 'purple'}
icons = {'Coffee Shop': 'coffee', 'Airport': 'plane','School': 'book', 'Vegan': 'apple' }

In [112]:
"""
locations.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=10, fill_color=colors[row['score']],popup=row['name'])
                                             .add_to(m), axis=1)"""


'\nlocations.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], \n                                              radius=10, fill_color=colors[row[\'score\']],popup=row[\'name\'])\n                                             .add_to(m), axis=1)'

In [113]:
map2 = folium.Map(location=[37.5579101647, -122.3221364075], tiles='CartoDB positron', zoom_start=11)

marker_cluster = folium.plugins.MarkerCluster().add_to(map2)

for index,row in locations.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]],
                popup = [row['name']],
                icon=folium.Icon(color=colors[row['score']], icon = icons[row['category']],
                icon_color='white', angle=0, prefix='fa')).add_to(marker_cluster)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: UserWarning: color argument of Icon should be one of: {'lightgray', 'gray', 'pink', 'green', 'darkgreen', 'white', 'orange', 'darkred', 'red', 'cadetblue', 'lightblue', 'black', 'lightgreen', 'darkblue', 'blue', 'beige', 'darkpurple', 'lightred', 'purple'}.
  if __name__ == '__main__':


# Companies collection

In [91]:
db, coll2 = connectCollection('companies','best')

In [92]:
best = db.best.find({ 
    "category":{
        "$eq": 'web'
    }})

In [93]:
locations_comp = pd.DataFrame(best)

In [94]:
locations_comp.drop(['coordinates','_id','id'], axis=1 ,inplace=True)

In [97]:
locations_comp.head()

,name,employees,year,category,money_raised,city,country,latitude,longitude,number offices near
0,Social Gaming Network,100.0,2011,web,17100000.0,Los Angeles,USA,37.446823,-122.161523,3
1,Burt,30.0,2009,web,3500000.0,Gothenburg,SWE,59.337864,18.036587,2
2,Inuvo,50.0,2009,web,4200000.0,Clearwater,USA,27.913011,-82.702449,2
3,Burt,30.0,2009,web,3500000.0,Gothenburg,SWE,59.337864,18.036587,2
4,Inuvo,50.0,2009,web,4200000.0,Clearwater,USA,27.913011,-82.702449,2


In [105]:
colors2 = {2: 'blue', 3: 'yellow', 6:'orange',7:'red'}
icons2= {'web': 'briefcase'}

In [106]:
for index,row in locations_comp.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]],
                popup = [row['name']],
                icon=folium.Icon(color=colors2[row['number offices near']], icon = icons2[row['category']],
                icon_color='white', angle=0, prefix='fa')).add_to(marker_cluster)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: UserWarning: color argument of Icon should be one of: {'lightgray', 'gray', 'pink', 'green', 'darkgreen', 'white', 'orange', 'darkred', 'red', 'cadetblue', 'lightblue', 'black', 'lightgreen', 'darkblue', 'blue', 'beige', 'darkpurple', 'lightred', 'purple'}.
  """


In [108]:
map2.save('mapNY.html')